# Выпускной проект: Проект по SQL

Описание проекта: 
    
Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Описание данных:

Таблица books содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Задания:
Посчитайте, сколько книг вышло после 1 января 2000 года;

Для каждой книги посчитайте количество обзоров и среднюю оценку;

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

#### Посчитайте, сколько книг вышло после 1 января 2000 года

Выведем первые 5 строк

In [4]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выведем те, которые по дате публикации вышли после 01.01.2020

In [5]:
query = '''SELECT book_id, title, publication_date
FROM books 
WHERE publication_date >= '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,publication_date
0,1,'Salem's Lot,2005-11-01
1,2,1 000 Places to See Before You Die,2003-05-22
2,3,13 Little Blue Envelopes (Little Blue Envelope...,2010-12-21
3,4,1491: New Revelations of the Americas Before C...,2006-10-10
4,5,1776,2006-07-04
...,...,...,...
816,994,World War Z: An Oral History of the Zombie War,2006-09-12
817,996,Wyrd Sisters (Discworld #6; Witches #2),2001-02-06
818,998,Year of Wonders,2002-04-30
819,999,You Suck (A Love Story #2),2007-01-16


821 строчка, получается 821 книга, выведем только количество книг

In [6]:
query = '''SELECT 
COUNT(book_id) as count_books
FROM books
WHERE publication_date >= '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count_books
0,821


821 книга вышла после 1 января 2000 года.

#### Для каждой книги посчитайте количество обзоров и среднюю оценку

Понадобятся таблицы books, ratings, reviews

In [7]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [8]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Для начала попробуем обьеденить books, ratings, reviews, и выведем столбцы books, book_id, review_id, rating

In [10]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT books.title, books.book_id, reviews.review_id, ratings.rating
FROM books
INNER JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,book_id,review_id,rating
0,'Salem's Lot,1,2.0,4
1,'Salem's Lot,1,1.0,4
2,'Salem's Lot,1,2.0,2
3,'Salem's Lot,1,1.0,2
4,'Salem's Lot,1,2.0,5
...,...,...,...,...
26175,Zen and the Art of Motorcycle Maintenance: An ...,1000,2790.0,2
26176,Zen and the Art of Motorcycle Maintenance: An ...,1000,2793.0,5
26177,Zen and the Art of Motorcycle Maintenance: An ...,1000,2792.0,5
26178,Zen and the Art of Motorcycle Maintenance: An ...,1000,2791.0,5


Получилась таблица с нужными столбцами, осталось подсчитать количество обзоров и среднюю оценку

In [11]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT books.title, books.book_id, COUNT(DISTINCT reviews.review_id) as COUNT, AVG(ratings.rating) as AVG
FROM books
INNER JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY books.title, books.book_id
ORDER BY COUNT DESC;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,book_id,count,avg
0,Twilight (Twilight #1),948,7,3.662500
1,Water for Elephants,963,6,3.977273
2,The Glass Castle,734,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,695,6,4.081081
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667
996,The Natural Way to Draw,808,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
998,Essential Tales and Poems,221,0,4.000000


Книга Twilight (Twilight #1) с самым большим количеством обзоров имеет среднюю оценку 3,66

#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

Понадобятся таблицы books и publishers

In [12]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [13]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Обьеденим books и publishers, выведем столбцы publisher и publisher_id, при условии где страниц будет больше 50

In [14]:
query = '''SELECT publishers.publisher, publishers.publisher_id
FROM publishers
JOIN books ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,publisher_id
0,Doubleday,93
1,Workman Publishing Company,336
2,HarperCollins Publishers,135
3,Vintage,309
4,Simon Schuster,268
...,...,...
987,Hartorch,147
988,Tor Books,297
989,Penguin Books,212
990,William Morrow,331


Подсчитаем количество книг, которое выпустило издание

In [15]:
query = '''SELECT publishers.publisher, publishers.publisher_id, COUNT(books.book_id) as COUNT
FROM publishers
JOIN books ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher_id
ORDER BY COUNT DESC'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,publisher_id,count
0,Penguin Books,212,42
1,Vintage,309,31
2,Grand Central Publishing,116,25
3,Penguin Classics,217,24
4,Ballantine Books,33,19
...,...,...,...
329,Ballantine Books (NY),34,1
330,Plaza y Janés,225,1
331,HarperCollinsPublishers,138,1
332,Random House: Modern Library,245,1


In [16]:
query = '''SELECT publishers.publisher, publishers.publisher_id, COUNT(books.book_id) as COUNT
FROM publishers
JOIN books ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher_id
ORDER BY COUNT DESC
LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,publisher_id,count
0,Penguin Books,212,42


Больше всего книг выпустило издание Penguin Books

#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

Понадобятся таблицы authors, books, ratings

In [17]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [18]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [19]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Для начала соеденим, выведем столбцы author, author_id, rating, books.book_id

In [20]:
query = '''SELECT authors.author, authors.author_id, books.book_id, ratings.rating
FROM authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con) 

,author,author_id,book_id,rating
0,Stephen King/Jerry N. Uelsmann,546,1,4
1,Stephen King/Jerry N. Uelsmann,546,1,2
2,Stephen King/Jerry N. Uelsmann,546,1,5
3,Patricia Schultz,465,2,3
4,Patricia Schultz,465,2,2
...,...,...,...,...
6451,Robert M. Pirsig,509,1000,4
6452,Robert M. Pirsig,509,1000,4
6453,Robert M. Pirsig,509,1000,5
6454,Robert M. Pirsig,509,1000,2


Определим автора с самой высокой средней оценкой книг, учтем так же условие, что бы оценок было больше 50

In [21]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT 
authors.author AS iauthor, 
books.book_id AS idbook, 
AVG(ratings.rating) AS AVG, 
COUNT(ratings.rating) AS COUNT
FROM authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY 
idbook,  
author
HAVING COUNT(ratings.rating) >= 50
ORDER BY AVG DESC
;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,iauthor,idbook,avg,count
0,J.K. Rowling/Mary GrandPré,302,4.414634,82
1,J.R.R. Tolkien,722,4.391892,74
2,J.K. Rowling/Mary GrandPré,299,4.287500,80
3,Markus Zusak/Cao Xuân Việt Khương,656,4.264151,53
4,J.K. Rowling/Mary GrandPré,300,4.246575,73
5,Louisa May Alcott,399,4.192308,52
6,J.K. Rowling/Mary GrandPré,301,4.186667,75
7,J.R.R. Tolkien,750,4.125000,88
8,Rick Riordan,779,4.080645,62
9,William Golding,405,3.901408,71


Сохраним полученую таблицу в ii, и уже по ней сгрупируем по автору и подсчитаем снова среднюю оценку

In [22]:
query = '''SELECT ii.iauthor AS author, AVG(ii.avg) AS avg
FROM (SELECT 
authors.author AS iauthor, 
books.book_id AS idbook, 
AVG(ratings.rating) AS AVG, 
COUNT(ratings.rating) AS COUNT
FROM authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY 
idbook,  
iauthor
HAVING COUNT(ratings.rating) >= 50
ORDER BY AVG DESC) AS ii
GROUP BY author
ORDER BY avg DESC
;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Наибольшую среднюю оценку 4.28 имеет автор J.K. Rowling/Mary GrandPré.

#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

Понадобятся таблицы reviews, ratings

Выведем имена пользователей, у которых оценок болье 48

In [23]:
query = '''SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(ratings) > 48;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,username
0,walter49
1,sfitzgerald
2,jennifermiller
3,andreaeaton
4,xdavis
5,vcarter
6,paul88
7,lesliegibbs
8,susan85
9,martinadam


Подсчитаем количество оценок у пользователей

In [24]:
query = '''SELECT COUNT(review_id)
FROM reviews
WHERE username IN 
(SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(ratings) > 48)
GROUP BY username;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,23
1,28
2,25
3,22
4,18
5,20
6,22
7,26
8,29
9,27


Теперь, нужно подсчитать среднее количество обзоров

In [25]:
query = '''SELECT AVG(review.review_count) AS avg
FROM (SELECT COUNT(review_id) AS review_count
FROM reviews
WHERE username IN 
(SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(ratings) > 48)
GROUP BY username) 
AS review;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров равняется 24

### Общий вывод

В хоже работы, было подсчитано, что 821 книга вышла после 1 января 2000 года; 

Для каждой книги подсчитали количество обзоров и среднюю оценку, где книга Twilight (Twilight #1) с самым большим количеством обзоров имеет среднюю оценку 3,66; 

Определили издательство, которое выпустило наибольшее число книг толще 50 страниц, это оказалось издетельсво Penguin Books; 

Определили автора с самой высокой средней оценкой книг, где наибольшую среднюю оценку 4.28 имеет автор J.K. Rowling/Mary GrandPré; 

Средее количество обзоров от пользователей, которые поставили больше 48 оценок, равняется 24.